In [1]:
import numpy as np
import math
import cv2
import tensorflow as tf

In [116]:
def bilinear_interpolation_ref(img, new_row_sz, new_col_sz):
    old_row_sz, old_col_sz = img.shape[:2]
    new_img = np.zeros(shape=(new_row_sz, new_col_sz), dtype=np.uint8)
    
    row_scale = (old_row_sz-1) / (new_row_sz-1)
    col_scale = (old_col_sz-1) / (new_col_sz-1)
#     print("scales", row_scale, col_scale)
    
    row_mapping = []
    col_mapping = []
    
    row_nearest_idxs = []
    col_nearest_idxs = []
    
    row_deltas = []
    col_deltas = []
    new_col_idx = 1
    
    tmp_points, tmp_new_pixels = [], []
    for new_row_idx in range(0, new_row_sz):
        for new_col_idx in range(0, new_col_sz):
            row_mapped_pos = row_scale * new_row_idx 
            row_mapping.append(row_mapped_pos)
            col_mapped_pos = col_scale * new_col_idx
            col_mapping.append(col_mapped_pos)
            
            row_nearest_idx = int(np.floor(row_mapped_pos))
#             row_nearest_idx = min(row_nearest_idx, old_row_sz - 2)
#             row_nearest_idx = max(row_nearest_idx, 1)
            
            row_nearest_idxs.append(row_nearest_idx)
            col_nearest_idx = int(np.floor(col_mapped_pos))
#             col_nearest_idx = min(col_nearest_idx, old_col_sz - 2)
#             col_nearest_idx = max(col_nearest_idx, 1)
            col_nearest_idxs.append(col_nearest_idx)
            
#             print("mapped postions", row_mapped_pos, col_mapped_pos, "-->", row_nearest_idx, col_nearest_idx)

            row_delta = row_mapped_pos - row_nearest_idx
            row_deltas.append(row_delta)
            col_delta = col_mapped_pos - col_nearest_idx
            col_deltas.append(col_delta)
#             print(new_row_idx, new_col_idx, "-->", row_nearest_idx, col_nearest_idx, "with deltas:", row_delta, col_delta)
            
            right_top_pixel = img[row_nearest_idx, col_nearest_idx] 
            
#             if row_nearest_idx == old_row_sz - 1:
#                 right_bottom_pixel = 0
#                 left_bottom_pixel = 0
#             elif col_nearest_idx == old_col_sz - 1:
#                 left_top_pixel = 0
#                 left_bottom_pixel = 0
#             else:
#                 right_bottom_pixel = img[row_nearest_idx+1, col_nearest_idx] 
#                 left_bottom_pixel = img[row_nearest_idx+1, col_nearest_idx+1]
#                 left_top_pixel = img[row_nearest_idx, col_nearest_idx+1] 
#                 left_bottom_pixel = img[row_nearest_idx+1, col_nearest_idx+1]
            
            y_idx = min(row_nearest_idx+1, old_row_sz-1)
            x_idx = min(col_nearest_idx+1, old_col_sz-1)
            right_bottom_pixel = img[y_idx, col_nearest_idx] 
            left_top_pixel = img[row_nearest_idx, x_idx] 
            left_bottom_pixel = img[y_idx, x_idx]
            
            tmp_points.append([row_nearest_idx, col_nearest_idx, y_idx, x_idx])
            
            new_pixel = right_top_pixel * (1 - row_delta)  * (1 - col_delta) + \
                        right_bottom_pixel  * (row_delta)  * (1 - col_delta) + \
                        left_top_pixel * (1 - row_delta)  * (col_delta) + \
                        left_bottom_pixel * (row_delta)  * (col_delta) 
            
            tmp_new_pixels.append(new_pixel)
            
            new_img[new_row_idx, new_col_idx] = new_pixel
    return new_img
    

In [ ]:
# // CalculateResizeScale determines the float scaling factor.
# inline float CalculateResizeScale(int64_t in_size, int64_t out_size,
#                                   bool align_corners) {
#   return (align_corners && out_size > 1)
#              ? (in_size - 1) / static_cast<float>(out_size - 1)
#              : in_size / static_cast<float>(out_size);
# }

# return (static_cast<float>(x) + 0.5f) * scale - 0.5f;

In [308]:
def do_bilinear_resize(img, new_size):
    '''
    Takes in a image with three channels and
    resizes to @param: new_size
    '''
    num_row_img, num_col_img, num_channels = img.shape
    resized_img = np.zeros((new_size[0], new_size[1], img.shape[-1]), dtype=np.uint8)
    num_row_resized_img, num_col_resized_img = resized_img.shape[:2]
    
    def get_nearest_idxs_with_deltas(old_sz, new_sz):
        # if (op_params.align_corners && output_height > 1) {
        # height_scale = static_cast<float>(input_height - 1) / (output_height - 1);}
#         scale = (old_sz -1 ) / (new_sz-1) # for align corner
        scale = (old_sz ) / (new_sz) # for half pixel centers
        print(scale)
        
        nearest_idxs, deltas = [], []
        for new_idx in range(0, new_sz):
            # mapped_position = scale * new_idx # align is True?
#             mapped_position = (new_idx + 0.5) * (scale) - 0.5 # half_pixel_centers: *scaled_value = (value + 0.5f) * scale - 0.5f;
            mapped_position = max((new_idx + 0.5) * (scale) - 0.5, 0) # half_pixel_centers: *scaled_value = (value + 0.5f) * scale - 0.5f;
            mapped_position = min(mapped_position, old_sz - 1)
            nearest_idx = math.floor(mapped_position)
            delta = mapped_position - nearest_idx
            nearest_idxs.append(nearest_idx)
            deltas.append(delta)
            
#             print("idx mapped: ", new_idx, "-->", nearest_idx, "delta:", delta)
        return nearest_idxs, deltas
    
#     print("rows")
    row_nearest_idxs, row_deltas = get_nearest_idxs_with_deltas(old_sz=num_row_img, new_sz=num_row_resized_img)
#     print("cols")
    col_nearest_idxs, col_deltas = get_nearest_idxs_with_deltas(old_sz=num_col_img, new_sz=num_col_resized_img)
    
    tmp_row_nearest_idxs, tmp_col_nearest_idxs = [], []
    tmp_row_deltas, tmp_col_deltas, tmp_points, new_pixels = [], [], [], []
    for channel_idx in range(0, num_channels):
        for row_idx in range(0, num_row_resized_img):
            for col_idx in range(0, num_col_resized_img):
                row_nearest_idx = row_nearest_idxs[row_idx]
                col_nearest_idx = col_nearest_idxs[col_idx]
                
                tmp_row_nearest_idxs.append(row_nearest_idx)
                tmp_col_nearest_idxs.append(col_nearest_idx)
                
                row_delta = row_deltas[row_idx]
                col_delta = col_deltas[col_idx]
                
                tmp_row_deltas.append(row_delta)
                tmp_col_deltas.append(col_delta)
                
                row_idx_next = row_nearest_idx + 1
                row_idx_next = min(row_idx_next, num_row_img - 1)
                col_idx_next = col_nearest_idx + 1
                col_idx_next = min(col_idx_next, num_col_img - 1)

                right_top_pixel = img[row_nearest_idx, col_nearest_idx, channel_idx].astype(np.float32) 
                right_bottom_pixel = img[row_idx_next, col_nearest_idx, channel_idx].astype(np.float32) 
                left_top_pixel = img[row_nearest_idx, col_idx_next, channel_idx].astype(np.float32) 
                left_bottom_pixel = img[row_idx_next, col_idx_next, channel_idx].astype(np.float32)
                
                tmp_points.append([row_nearest_idx, col_nearest_idx, row_idx_next,  col_idx_next])
                
                new_pixel = (right_top_pixel * (1 - row_delta)  * (1 - col_delta)).astype(np.float32) + \
                            (right_bottom_pixel  * (row_delta)  * (1 - col_delta)).astype(np.float32) + \
                            (left_top_pixel * (1 - row_delta)  * (col_delta)).astype(np.float32) + \
                            (left_bottom_pixel * (row_delta)  * (col_delta)).astype(np.float32)

                new_pixels.append(new_pixel)
                resized_img[row_idx, col_idx, channel_idx] = new_pixel
    return resized_img

In [309]:
img = cv2.imread("./lena.png")
img = img[:6, :6, 0]
new_h, new_w = 14, 14
ref_img = cv2.resize(img, (new_w, new_h), interpolation = cv2.INTER_LINEAR)

In [310]:
m1_img = bilinear_interpolation_ref(img=img, new_row_sz=new_h, new_col_sz=new_w)# .astype(np.uint8)
m2_img = do_bilinear_resize(img=img[..., np.newaxis], new_size=(new_h, new_w))
# mask = m1_img.reshape(-1) != m2_img.reshape(-1)
# m1_img.reshape(-1)[mask], m2_img.reshape(-1)[mask]


# upsampling is more consistent with opencv

0.42857142857142855
0.42857142857142855


In [311]:
img[np.newaxis, ...].shape

(1, 6, 6)

In [312]:
img

array([[125, 125, 133, 128, 120, 116],
       [125, 125, 133, 128, 120, 116],
       [125, 125, 133, 128, 120, 116],
       [125, 125, 133, 128, 120, 116],
       [125, 125, 133, 128, 120, 116],
       [123, 123, 113, 111, 120, 115]], dtype=uint8)

In [313]:
ref_img

array([[125, 125, 125, 125, 128, 132, 131, 129, 127, 123, 120, 118, 117,
        116],
       [125, 125, 125, 125, 128, 132, 131, 129, 127, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 132, 131, 129, 127, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 132, 132, 129, 127, 123, 120, 118, 117,
        116],
       [125, 125, 125, 125, 128, 132, 131, 129, 127, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 132, 131, 129, 127, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 132, 131, 129, 127, 123, 120, 118, 117,
        116],
       [125, 125, 125, 125, 128, 132, 131, 129, 127, 123, 120, 118, 117,
        116],
       [125, 125, 125, 125, 128, 132, 131, 129, 127, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 132, 131, 129, 127, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 132, 132, 129, 127, 123, 120, 118, 117,
        116],
       [124, 124, 124, 124, 124, 124, 123, 

In [314]:
m2_img.reshape(new_h, new_w)

array([[125, 125, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 124, 124, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 124, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 124, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 124, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 124, 124, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 124, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [124, 124, 124, 124, 124, 124, 123, 

In [315]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [316]:
t = tf.constant(img[...,np.newaxis])   # tensorflow needs a batch axis and a channel axis
tf_ref = tf.image.resize(
    t, size=(new_h, new_w),  preserve_aspect_ratio=False, 
    antialias=False, name=None
).reshape(new_h, new_w).astype(np.uint8).numpy()
tf_ref

array([[125, 125, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 131, 131, 129, 126, 123, 120, 118, 116,
        116],
       [124, 124, 124, 124, 124, 124, 123, 

In [317]:
ref_img

array([[125, 125, 125, 125, 128, 132, 131, 129, 127, 123, 120, 118, 117,
        116],
       [125, 125, 125, 125, 128, 132, 131, 129, 127, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 132, 131, 129, 127, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 132, 132, 129, 127, 123, 120, 118, 117,
        116],
       [125, 125, 125, 125, 128, 132, 131, 129, 127, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 132, 131, 129, 127, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 132, 131, 129, 127, 123, 120, 118, 117,
        116],
       [125, 125, 125, 125, 128, 132, 131, 129, 127, 123, 120, 118, 117,
        116],
       [125, 125, 125, 125, 128, 132, 131, 129, 127, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 132, 131, 129, 127, 123, 120, 118, 116,
        116],
       [125, 125, 125, 125, 128, 132, 132, 129, 127, 123, 120, 118, 117,
        116],
       [124, 124, 124, 124, 124, 124, 123, 

In [319]:
mask = m2_img.reshape(new_h, new_w) == tf_ref

In [324]:
mask

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True, False, False,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True, False,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True, False,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True, False,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True],
       [ True, False, False,  True,  True,  True,  True,  True,  True,
         

In [322]:
m2_img.reshape(new_h, new_w)[~mask]

array([124, 124, 124, 124, 124, 124, 124, 124], dtype=uint8)

In [323]:
tf_ref[~mask]

array([125, 125, 125, 125, 125, 125, 125, 125], dtype=uint8)

In [ ]:
%%timeit
do_bilinear_resize(img, new_size=(12, 12));

In [ ]:
%%timeit
bilinear_interpolation_ref(img=img, new_row_sz=12, new_col_sz=12).astype(np.uint8)

## Bilinear interpolation as convolution

In [ ]:

def get_upsampling_conv_properties(N, R):
    '''
    Returns parameters for upsampling convolution:
        # input, N X N --> output, R X R
        # 1. S := (N - 1) /  gcd(N-1, R-1)
        # 2. k := (R - 1) /  gcd(N-1, R-1)
        # 3. Convolution(kxk, stride=S, lhs_dilation=k, padding=k-1)
    '''
    def gcd(a,b):
        if(b==0):
            return a
        else:
            return gcd(b,a%b)
    
    _gcd_N_R = gcd(N, R)
    S = (N - 1) // _gcd_N_R
    K = (R - 1) // _gcd_N_R

    kernel_sz = K * 2 - 1
    padding = K-1
    lhs_dillation = K
    return K, kernel_sz, padding, lhs_dillation

def insert_zeros_2d(a, N=1, zero_point=0):
    '''
    Expand input 2d array for specified dillation
    '''
    out = np.ones( (N+1)*np.array(a.shape)-N, dtype=a.dtype) * zero_point
    out[::N+1,::N+1] = a
    return out

def get_bilinear_1D_kernel(n:int):
    '''
    Builds bilinear 1d kernel
    '''
    kernel_sz = n * 2 - 1
    kernel = np.zeros(shape=(kernel_sz,))
    for idx in range(0, n):
        v = idx + 1.0
        kernel[idx] = v
        kernel[n * 2 - 2 - idx] = v
    return kernel

def get_bilinear_2D_kernel(n:int):
    kernel_1d = get_bilinear_1D_kernel(n)
    kernel_2d = np.zeros((kernel_1d.shape[0], kernel_1d.shape[0]), dtype=np.int)
    kernel_2d[:,:] = kernel_1d
    kernel_2d = kernel_2d * kernel_2d.T
    return kernel_2d

In [ ]:
get_bilinear_1D_kernel(n = 3)

In [ ]:
arr = np.zeros((5,5))
arr[:,:] = get_bilinear_1D_kernel(n = 3)

In [ ]:
arr = np.array([0, 3, 6, 9, 12, 15, 18, 21, 24]).reshape(3,3)
expanded = insert_zeros_2d(a=arr, N=2, zero_point=0)
expanded = np.pad(expanded, pad_width=2)
expanded

In [ ]:
kernel = np.array([1,2,3,2,1,2,4,6,4,2,3,6,9,6,3,2,4,6,4,2,1,2,3,2,1]).reshape(5, 5)

In [ ]:
do_bilinear_resize(img=arr[..., np.newaxis], new_size=(4,4)).reshape(4,4)

In [ ]:
from numpy.lib.stride_tricks import as_strided

def conv2d(a, b, s=1):
    Hout = (a.shape[1] - b.shape[0]) // s + 1
    Wout = (a.shape[2] - b.shape[1]) // s + 1
    Stride = (a.strides[0], a.strides[1] * s, a.strides[2] * s, a.strides[1], a.strides[2], a.strides[3])

    a = as_strided(a, (a.shape[0], Hout, Wout, b.shape[0], b.shape[1], a.shape[3]), Stride)

    return np.tensordot(a, b, axes=3)

# test
conv2d(a=expanded.reshape(1,11,11,1), b=kernel.reshape(5,5,1), s=2) / 9

In [ ]:
kernel_1d = get_bilinear_1D_kernel(n = 3); kernel_1d.reshape(1,1,5,1).T.shape

In [ ]:
arr1 = conv2d(a=expanded.reshape(1,11,11,1), b=kernel_1d.reshape(1, 5, 1), s=2)
arr2 = conv2d(a=arr1[..., np.newaxis], b=kernel_1d.reshape(1, 1, 5).T, s=2)

In [ ]:
arr2

In [ ]:
# TODO: Decompose and see

+ OpenCV resize() result is wrong - https://stackoverflow.com/a/43701718/6561141
+ [good explaination] Resize an image with bilinear interpolation without imresize https://stackoverflow.com/a/26143655/6561141
+  Richard Alan Peters' II digital image processing slides on interpolation - https://ia802707.us.archive.org/23/items/Lectures_on_Image_Processing/EECE_4353_15_Resampling.pdf
+ Understanding Bilinear Image Resizing https://chao-ji.github.io/jekyll/update/2018/07/19/BilinearResize.html
+ bilinear interpolation as convolution - https://github.com/tensorflow/tensorflow/blob/cddf9415564b16c2bc234df68d3eb44fc8689dae/tensorflow/compiler/tf2xla/kernels/image_resize_ops.cc
+ https://jricheimer.github.io/tensorflow/2019/02/11/resize-confusion/
+ https://www.cs.toronto.edu/~guerzhoy/320/lec/upsampling.pdf
+ tf resize vs opencv resize - https://github.com/tensorflow/tensorflow/issues/6720
+ tf code resize bilinear - https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/kernels/internal/reference/resize_bilinear.h
+ tf code resize call - https://github.com/tensorflow/tensorflow/blob/b36436b087bd8e8701ef51718179037cccdfc26e/tensorflow/python/ops/image_ops_impl.py
+ tf2xla bi linear implementation as conv - https://github.com/tensorflow/tensorflow/blob/cddf9415564b16c2bc234df68d3eb44fc8689dae/tensorflow/compiler/tf2xla/kernels/image_resize_ops.cc
+ https://chao-ji.github.io/jekyll/update/2018/07/19/BilinearResize.html
+ https://stackoverflow.com/questions/26142288/resize-an-image-with-bilinear-interpolation-without-imresize
+ 

+ https://www.tensorflow.org/api_docs/python/tf/image/resize
+ https://github.com/tensorflow/tensorflow/blob/v2.6.0/tensorflow/python/ops/image_ops_impl.py#L1547-L1723
+ https://github.com/tensorflow/tensorflow/blob/v2.6.0/tensorflow/python/ops/init_ops_v2.py
+ https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/kernels/image/resize_bilinear_op.cc
+ https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/util/image_resizer_state.h
+ https://github.com/tensorflow/tensorflow/issues/24426

+ https://jricheimer.github.io/tensorflow/2019/02/11/resize-confusion/
+ https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/kernels/internal/reference/resize_bilinear.h
+ https://stackoverflow.com/questions/26142288/resize-an-image-with-bilinear-interpolation-without-imresize